In [1]:
import datapane as dp
import altair as alt
import pandas as pd
from sqlalchemy import create_engine, text
import random

## Load the data

Our data is in an SQLite database, and our wrangling operations are offered by `pandas`, so let's [get our data into](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#sql-queries) a `DataFrame`.

In [2]:
engine = create_engine("sqlite:///assets/kaggle_survey.db")
conn = engine.connect()
data = pd.read_sql_table("data", conn)
dp.DataTable(data.head())

Let's retrieve the descriptions too.

In [3]:
descriptions = pd.read_sql_table("descriptions", conn)
dp.DataTable(descriptions)

In [4]:
segment_mapping = {
    "Roles": "Q5",
    "Industry": "Q20",
    "DS Team Size": "Q22",
    "Primary Tool": "Q41",
}

## Word cloud function

In [5]:
def word_cloud(df):
    words_and_counts = df.melt()["value"].dropna().value_counts()
    words_and_counts = pd.DataFrame(words_and_counts).rename_axis().reset_index()
    words_and_counts.columns = ["word", "count"]

    def shuffled_range(n):
        return random.sample(range(n), k=n)

    n = len(words_and_counts)
    x = shuffled_range(n)
    y = shuffled_range(n)

    word_cloud_data = words_and_counts.assign(x=x, y=y)

    base = (
        alt.Chart(word_cloud_data)
        .encode(x=alt.X("x:O", axis=None), y=alt.Y("y:O", axis=None))
        .configure(background="#eef2ff")
    )

    word_cloud = (
        base.mark_text(baseline="middle")
        .encode(
            text="word:N",
            color=alt.Color("count:Q", scale=alt.Scale(scheme="purpleblue")),
            size=alt.Size("count:Q", legend=None, scale=alt.Scale(range=[20, 50])),
        )
        .configure_view(strokeWidth=0)
    )

    return dp.Plot(word_cloud)

Test our word cloud on all the question 9 responses:

In [6]:
word_cloud(data.filter(like="Q9_Part"))

## Plotting segments

In [7]:
def plot_segment_distribution(df, segment_name):
    
    segments = df[segment_mapping[segment_name]]
    counts = pd.DataFrame(segments.value_counts()).rename_axis().reset_index()
    counts.columns = [segment_name, "counts"]
    counts

    fig = (
        alt.Chart(counts)
        .mark_bar()
        .encode(
            x=alt.X(counts.columns[0], sort="-y", axis=alt.Axis(labelAngle=-45)),
            y="counts",
            color=alt.Color(
                segment_name, scale=alt.Scale(scheme="rainbow"), legend=None
            ),
        )
    )

    return dp.Plot(fig)

Test plotting the distribution of the Roles segment

In [8]:
plot_segment_distribution(data, "Roles")

## Filtering with a form

Let's allow our user to filter the survey data and generate a word cloud themselves.

First, we'll look towards filtering by programming language. These are in columns with a `Q7_Part_` prefix.

In [9]:
programming_languages = (
    data.filter(like="Q7_Part_").melt().dropna().drop_duplicates().reset_index(drop=True)
)

programming_languages = pd.concat(
    [
        programming_languages,
        pd.DataFrame([["*", "All"]], columns=programming_languages.columns),
    ]
)

dp.Table(programming_languages)

## Overall summary

In [10]:
stats_group = dp.Group(
    dp.BigNumber(heading="Participants", value=len(data)),
    dp.BigNumber(heading="Segments", value=len(segment_mapping)),
)

stats_group

In [17]:
def f(params):
    filtered_data = data

    if params["programming_language"] != "All":
        programming_language_column = programming_languages[
            programming_languages.value == params["programming_language"]
        ].variable.item()
        filtered_data = filtered_data[
            filtered_data[programming_language_column] == params["programming_language"]
        ]

    segment_plots = dp.Group(
        blocks=[
            plot_segment_distribution(filtered_data, segment)
            for segment in segment_mapping.keys()
        ]
    )

    choice_id_looking = {
        "What machine learning framework do you use?": "Q9",
        "What programming language do you use?": "Q7",
    }

    word_cloud_plot = word_cloud(
        filtered_data.filter(like=choice_id_looking[params["choice"]])
    )

    return dp.Group(
        f"## {params['choice']}",
        word_cloud_plot,
        "## Breakdown by segment",
        segment_plots,
    )


form = dp.Form(
    on_submit=f,
    target="output",
    label="Filters",
    controls=[
        dp.Choice(
            "programming_language",
            label="Programming language",
            initial="All",
            options=list(programming_languages.value),
        ),
        dp.Choice(
            "choice",
            label="Word cloud: survey question",
            initial="What programming language do you use?",
            options=[
                "What machine learning framework do you use?",
                "What programming language do you use?",
            ],
        ),
    ],
)


v = dp.Blocks(
    "# Kaggle Survey 2021",
    dp.Group(form, stats_group, columns=2),
    dp.Empty(name="output"),
)

In [ ]:
dp.serve_app(v)

Running serve_app on a static report - do you want to use save_report or upload_report instead?
Please install Jupyter Widgets for an improved experienced with apps in Notebook environments: https://ipywidgets.readthedocs.io/en/latest/user_install.html
